In [1]:
import numpy as np 
import os
import cv2
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.applications.vgg16 import VGG16, preprocess_input

from keras import backend as K
from keras.utils import plot_model
n=224
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
from skimage.measure import label, regionprops
import skimage.io as io
import skimage.transform as trans
from keras.initializers import Constant

from matplotlib import pyplot as plt
%matplotlib inline

from skimage.morphology import disk
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import jaccard_similarity_score

C:\Users\kamru\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def FCN8(nClasses, input_height, input_width):

    #-----------------------------------------Define Input and Create VGG Layers---------------------------------------
    img_input = Input(shape=(input_height, input_width, 3))
    vgg = VGG16( weights='imagenet', include_top=False, input_tensor=img_input)


    #----------------------------------------FCN8 for the segmentation-------------------------------------------------
    vgg_out = Conv2D(filters=4096, kernel_size=(7, 7), padding="same", activation="relu", name="fc6")(vgg.output)
#     o = Dropout(rate=0.5)(o)
    
    vgg_out_conv_1 = Conv2D(filters=4096, kernel_size=(1, 1), padding="same", activation="relu", name="fc7")(vgg_out)
#     o = Dropout(rate=0.5)(o)

    vgg_out_conv_2 = Conv2D(filters=nClasses, kernel_size=(1, 1), padding="same", activation="relu", kernel_initializer="he_normal",
               name="score_fr")(vgg_out_conv_1)

    vgg_out_convTR = Conv2DTranspose(filters= nClasses,
                                     kernel_size=(2, 2), 
                                     strides=(2, 2), 
                                     padding="valid", 
                                     activation=None,
#                                      kernel_initializer=Constant(bilinear_upsample_weights(2, nClasses)),
                                     name="score2")(vgg_out_conv_1)

    # Conv to be applied on Pool4
    skip_con1 = Conv2D(nClasses, kernel_size=(1, 1), padding="same", activation=None, kernel_initializer="he_normal",
                       name="score_pool4")(vgg.get_layer("block4_pool").output)
    Skip_1 = add(inputs=[skip_con1, vgg_out_convTR])

    Skip_temp = Conv2DTranspose(nClasses, 
                                kernel_size=(2, 2), 
                                strides=(2, 2), 
                                padding="valid", 
                                activation=None,
#                                 kernel_initializer=Constant(bilinear_upsample_weights(2, nClasses)),
                                name="score4")(Skip_1)

    ###
    skip_con2 = Conv2D(nClasses, kernel_size=(1, 1), padding="same", activation=None, kernel_initializer="he_normal",
                       name="score_pool3")(vgg.get_layer("block3_pool").output)
    Skip_2 = add(inputs=[skip_con2, Skip_temp])

    #####
    fINALlAYER = Conv2DTranspose(nClasses, 
                                 kernel_size=(8, 8), 
                                 strides=(8, 8),
                                 padding="valid", 
                                 activation='relu', 
#                                  kernel_initializer=bilinear_upsample_weights(8, nClasses),
                                 name="upsampling")(Skip_2)

    
    

    vgg_Base = vgg
    
    deconv_1 = UpSampling2D((2, 2))(vgg_Base.output)
    deconv_1 = concatenate([vgg_Base.get_layer(name="block4_pool").output, deconv_1], axis=-1)
    deconv_1 = Conv2D(512, (3, 3), padding="same")(deconv_1)
    deconv_1 = BatchNormalization()(deconv_1)

    deconv_2 = UpSampling2D((2, 2))(deconv_1)
    deconv_2 = concatenate([vgg_Base.get_layer(name="block3_pool").output, deconv_2], axis=-1)
    deconv_2 = Conv2D(256, (3, 3), padding="same")(deconv_2)
    deconv_2 = BatchNormalization()(deconv_2)

    deconv_3 = UpSampling2D((2, 2))(deconv_2)
    deconv_3 = concatenate([vgg_Base.get_layer(name="block2_pool").output, deconv_3], axis=-1)
    deconv_3 = Conv2D(128, (3, 3), padding="same")(deconv_3)
    deconv_3 = BatchNormalization()(deconv_3)

    deconv_4 = UpSampling2D((2, 2))(deconv_3)
    deconv_4 = concatenate([vgg_Base.get_layer( name="block1_pool").output, deconv_4], axis=-1)
    deconv_4 = Conv2D(64, (3, 3), padding="same")(deconv_4)
    deconv_4 = BatchNormalization()(deconv_4)

    deconv_5 = UpSampling2D((2, 2))(deconv_4)
    deconv_5 = Conv2D(64, (3, 3), padding="same")(deconv_5)
    deconv_5 = BatchNormalization()(deconv_5)

    deconv_6 = Conv2D(nClasses, (1, 1), padding="same")(deconv_5)
    deconv_6 = BatchNormalization()(deconv_6)
    deconv_6 = Activation("relu")(deconv_6)

#     deconv_7 = Reshape((-1, nClasses))(deconv_6)
#     deconv_7 = Activation("softmax")(deconv_7)

#     conv10 = Conv2D(1, 1, activation = 'sigmoid')(deconv_6)

    
    deco6=Average()([deconv_6, fINALlAYER])
    
    conv = Conv2D(1, 1, activation = 'sigmoid')(deco6)
    
    modelUNET = Model(input = img_input, output = conv)


    for layer in modelUNET.layers[:26]:
        layer.trainable = False 


    return modelUNET



In [3]:
def adjustData(img, mask):
# #     img = preprocess_input(img)
#     img2= np.zeros_like(img)
#     for i in range(len(img[:,1,1,1])):
#         image=img[i,:,:,:]
# #         print(image.dtype)
        
#         im=preprocess_input(image)
#         grayVGG=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
# #         grayVGG=grayVGG.astype(np.uint8)
        
#         b,g,r=cv2.split(image)
        
#         gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
# #         print(b.shape)
# #         print(b.dtype)
        
# #         print(gray.shape)
# #         print(gray.dtype)
        
        
# #         print(grayVGG.shape)
# #         print(grayVGG.dtype)
        
#         sytheticImage=cv2.merge((b,grayVGG,gray))
        
# #         print(sytheticImage.shape)
       
        
#         sytheticImage = sytheticImage/sytheticImage.max()
        
        
#         img2 [i,:,:,:]=sytheticImage
        
    img = preprocess_input(img)
    img = img/img.max() 
    mask = mask/mask.max() 

    return (img, mask)

In [4]:
def trainGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode = "rgb",
                    mask_color_mode = "grayscale", image_save_prefix  = "image", mask_save_prefix  = "mask",
                    save_to_dir = None, target_size = (n,n), seed = 1):
    
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = 'rgb',
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)

    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)

    
    train_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask)
        yield (img,mask)

In [5]:
def ValGenerator(batch_size, val_path, image_folder, mask_folder, target_size = (n,n), seed = 1):
    
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    
    image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator()
    
    image_generator = image_datagen.flow_from_directory(
        val_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = 'rgb',
        target_size = target_size,
        batch_size = batch_size,
        seed = seed)

    
    mask_generator = mask_datagen.flow_from_directory(
        val_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = target_size,
        batch_size = batch_size,
        seed = seed)

    
    val_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in val_generator:
        img,mask = adjustData(img,mask)
        yield (img,mask)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score


def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, K.sigmoid(y_pred)) + dice_loss(y_true, K.sigmoid(y_pred))

In [ ]:
data_gen_args = dict(rotation_range=10,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=0.1,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

CurrentDirectory=os.getcwd()

TrainGen= trainGenerator(batch_size=5, train_path=CurrentDirectory+'/Data', image_folder='TrainImage2', mask_folder='TrainGT2',
                       aug_dict=data_gen_args, image_color_mode = "grayscale", mask_color_mode = "grayscale",
                       image_save_prefix  = "image", mask_save_prefix  = "mask", save_to_dir = CurrentDirectory+'/Aug/',
                       target_size = (n,n), seed = 1)


TestGen= ValGenerator(batch_size=5, val_path=CurrentDirectory+'/Data', image_folder='TestImage3', 
                      mask_folder='TestGT3', target_size = (n,n), seed = 1)

model = FCN8(2, n, n)

model.compile(optimizer = 'adadelta',
              loss = dice_loss, 
              metrics = [dice_coef])

plot_model(model, show_shapes=True, to_file='model_unet.png')
model.summary()

model_checkpoint = ModelCheckpoint('modelSaved.hdf5', monitor='val_loss', verbose=1, save_best_only=True)

history=model.fit_generator(TrainGen, 
                            steps_per_epoch=400, 
                            epochs=60, 
                            verbose=1, 
                            validation_data= TestGen, 
                            validation_steps=30, 
                            callbacks=[model_checkpoint])


C:\Users\kamru\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

Epoch 1/60
Found 150 images belonging to 1 classes.
Found 1998 images belonging to 1 classes.
Found 150 images belonging to 1 classes.
Found 1998 images belonging to 1 classes.
400/400 [==============================] - 610s 2s/step - loss: 0.4059 - dice_coef: 0.6591 - val_loss: 0.2837 - val_dice_coef: 0.7324

Epoch 00001: val_loss improved from inf to 0.28368, saving model to modelSaved.hdf5
Epoch 2/60
400/400 [==============================] - 593s 1s/step - loss: 0.2283 - dice_coef: 0.7818 - val_loss: 0.2747 - val_dice_coef: 0.7382

Epoch 00002: val_loss improved from 0.28368 to 0.27467, saving model to modelSaved.hdf5
Epoch 3/60
400/400 [==============================] - 596s 1s/step - loss: 0.2082 - dice_coef: 0.8000 - val_loss: 0.2534 - val_dice_coef: 0.7575

Epoch 00003: val_loss improved from 0.27467 to 0.25340, saving model to modelSaved.hdf5
Epoch 4/60
400/400 [==============================] - 605s 2s/step - loss: 0.1989 - dice_coef: 0.8080 - val_loss: 0.4191 - val_dice_coef

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid('on')
plt.show()


# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.grid('on')
plt.show()

In [ ]:
def dice(im1, im2, empty_score=1.0):
    
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

In [ ]:
def jaccard(im1, im2, empty_score=1.0):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)

    return intersection.sum() / float(union.sum())

In [ ]:
dicee=[]
JI=[]

In [ ]:
testGene = CurrentDirectory+'/ISIC-2017_Validation_Data (2)/'
imagePath = glob.glob(testGene+"*.jpg")
imagePath.sort()

GTPath= CurrentDirectory+'/ISIC-2017_Validation_Part1_GroundTruth/'
GT = glob.glob(GTPath+"*.png")
GT.sort()


MaskSavePath= CurrentDirectory+'/Mask/'

target_size = (n,n)

kernel=disk(7)
kernel2=disk(8)

model = FCN8(2, n, n)
model.load_weights("modelSaved.hdf5")

for imageName, gg in zip(imagePath,GT):
    img = cv2.imread(imageName,-1)
    sss=img.copy()
    
    filename, file_extension = os.path.splitext(imageName)  
    img = trans.resize(img,target_size)
    
    path0=MaskSavePath+filename[-12:]+'_org'+file_extension
    sss = cv2.resize(sss, (n, n)) 
    cv2.imwrite(path0,sss)
    
    
#     img = preprocess_input(img)
    img=np.expand_dims(img, axis=0)
    Pr=model.predict(img,verbose=0)
    Pr=Pr.reshape(n,n)
    path=MaskSavePath+filename[-12:]+'_pred'+file_extension
    cv2.imwrite(path,255*Pr)
    
    
    one=255*Pr


    one=one.astype('uint8')

    the,one=cv2.threshold(one,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)


#     one=cv2.erode(one,kernel,iterations = 1)
    
#     labelImage=label(one)
#     regions=regionprops(labelImage)
#     Area=[]
#     for region in range(len(regions)):
#         maxThreshold=regions[region].area,
#         Area.append(maxThreshold)
#     X= sorted( [(x,i) for (i,x) in enumerate(Area)], reverse=True )[:2]
    
#     index=np.array(X)
#     V=index.T[1].tolist()
# #     one[labelImage!=(V[0]+1)]=0,

    
#     one=cv2.dilate(one,kernel2,iterations = 1)
    
    path2=MaskSavePath+filename[-12:]+'_mask'+file_extension
    cv2.imwrite(path2,one)
    
    
    imGT=cv2.imread(gg,-1)
    imGT = cv2.resize(imGT, (n, n)) 
    tttt=cv2.merge((one,one,one))
    tttt[:,:,0]=0
    tttt[:,:,2]=0


#     plt.figure()
    plt.imshow(imGT, 'gray', interpolation='none')
    plt.imshow(tttt, 'jet', interpolation='none', alpha=0.3)
    plt.xticks([])
    plt.yticks([])
    path3=MaskSavePath+filename[-12:]+'_overlay'+file_extension
    plt.savefig(path3, bbox_inches='tight')
    plt.clf()
    
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (5,50)
    fontScale              = 1
    fontColor              = (0,0,255)
    lineType               = 2
    
    dd=dice(imGT,one)
    ji=jaccard(imGT,one)
    dicee.append(dd)
    JI.append(ji)
    
    imageee=cv2.imread(path3,-1)
    cv2.putText(imageee,str(dd), 
    bottomLeftCornerOfText, 
    font, 
    fontScale,
    fontColor,
    lineType)
    path4=MaskSavePath+filename[-12:]+'_dice'+file_extension
    cv2.imwrite(path4,imageee)
    
print("DSC = %0.4f (+/-%0.04f)" % (np.mean(dicee)*100, np.std(dicee)*100))
# print("Intersection over Union = %0.4f (+/-%0.04f)" % (np.mean(IU)*100, np.std(IU)*100))
# print("Sensitivity = %0.4f (+/-%0.04f)" % (np.mean(Sensitivity)*100, np.std(Sensitivity)*100))
# print("Specificity = %0.4f (+/-%0.04f)" % (np.mean(Specificity)*100, np.std(Specificity)*100))
# print("Accuracy = %0.8f (+/-%0.04f)" % (np.mean(Accuracy)*100, np.std(Accuracy)*100))
# print("Balanced Accuracy = %0.4f (+/-%0.04f)" % (np.mean(BalancedAccuracy)*100, np.std(BalancedAccuracy)*100))
# print("AUC = %0.4f (+/-%0.04f)" % (np.mean(ROC_AUC)*100, np.std(ROC_AUC)*100))
print("JI = %0.4f (+/-%0.04f)" % (np.mean(JI)*100, np.std(JI)*100))